In [ ]:
pip install snowflake-connector-python

In [304]:
import pandas as pd

In [305]:
import snowflake.connector as sc

In [306]:
def snwdb_conn():
    try:
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            )
        cur = conn.cursor()
        db_creation = cur.execute('create database if not exists COVID_DATA')
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            database = 'COVID_DATA'
            )
        cur = conn.cursor()
        schema_creation = cur.execute('CREATE SCHEMA IF NOT EXISTS STAGE')
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            database = 'COVID_DATA',
            schema = 'STAGE'
            )
        cur = conn.cursor()
        return conn,cur
    except sc.errors.Error as e:
        print(e,' unable to establish connection with SNW-DB')

In [307]:
connection,cursor = snwdb_conn()

In [308]:
create_country_dim_table = 'create or replace TABLE COVID_DATA.STAGE.COUNTRY_DIM (COUNTRY_NAME VARCHAR(255));'
cursor.execute(create_country_dim_table)

In [309]:
create_state_dim_table = 'create or replace TABLE COVID_DATA.STAGE.STATE_DIM (REGION_NAME VARCHAR(255),LAT VARCHAR(60),LON VARCHAR(60));'
cursor.execute(create_state_dim_table)

In [310]:
create_who_dim_table = 'create or replace TABLE COVID_DATA.STAGE.WHO_DIM (REGION_NAME VARCHAR(255));'
cursor.execute(create_who_dim_table)

In [340]:
create_CASES_DATA_FACT_table = 'create or replace TABLE COVID_DATA.COVID_DATASET_2019.CASES_DATA_FACT (COUNTRY_ID NUMBER(38,0) NOT NULL,STATE_ID NUMBER(38,0),WREGION_ID NUMBER(38,0) NOT NULL,DATE_ID NUMBER(38,0) NOT NULL,CONFIRMED NUMBER(38,0),DEATHS NUMBER(38,0),RECOVERED NUMBER(38,0),CREATED_DATE TIMESTAMP DEFAULT CURRENT_DATE(),CREATED_BY VARCHAR(50) DEFAULT CURRENT_USER(),foreign key (COUNTRY_ID) references COVID_DATA.COVID_DATASET_2019.COUNTRY_DIM(COUNTRY_ID),foreign key (STATE_ID) references COVID_DATA.COVID_DATASET_2019.STATE_DIM(STATE_ID),foreign key (WREGION_ID) references COVID_DATA.COVID_DATASET_2019.WHO_DIM(WREGION_ID),foreign key (DATE_ID) references COVID_DATA.COVID_DATASET_2019.DATE_DIM(DATE_ID));'
cursor.execute(create_CASES_DATA_FACT_table)

In [312]:
import csv

In [313]:
file_data = pd.read_csv('covid_19_clean_complete - Copy.csv')

In [314]:
file_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,Andhra Pradesh,India,13.8,12,28-02-2021,20,10,1,9,Asia


In [315]:
country_data = file_data[['Country/Region']].drop_duplicates().reset_index(drop = True).replace(r'[*]','',regex = True)

In [316]:
country_data[country_data['Country/Region'].str.contains(r'[()]')]

,Country/Region


In [317]:
state_data = file_data.loc[~file_data['Province/State'].isna()][['Province/State','Lat','Long']].drop_duplicates().reset_index(drop=True)

In [318]:
state_data

,Province/State,Lat,Long
0,Andhra Pradesh,13.8,12


In [319]:
who_data = file_data[['WHO Region']].drop_duplicates().reset_index(drop = True)

In [320]:
Date_dim = pd.to_datetime(file_data['Date']).drop_duplicates().reset_index(drop = True).to_frame()

C:\Users\polam\AppData\Local\Temp\ipykernel_36360\2149730354.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  Date_dim = pd.to_datetime(file_data['Date']).drop_duplicates().reset_index(drop = True).to_frame()


In [321]:
Date_dim['Month'] = Date_dim['Date'].dt.month_name()

In [322]:
Date_dim['Day'] = Date_dim['Date'].dt.day_name()

In [323]:
Date_dim['Year'] = Date_dim['Date'].dt.year

In [324]:
Date_dim['Month_Number'] = Date_dim['Date'].dt.month

In [325]:
Date_dim['Dat_Number'] = Date_dim['Date'].dt.day

In [327]:
call_trunucate_sp = cursor.execute('CALL TRUNCATE_STAGING_TABLES()')

In [335]:
insert_country_dim = 'insert into COVID_DATA.STAGE.country_dim(country_name) values(%s)'

In [329]:
for row,values in country_data.iterrows():
    cursor.execute(insert_country_dim,list(values))

In [336]:
insert_who_dim = 'insert into COVID_DATA.STAGE.who_dim(REGION_NAME) values(%s)'

In [331]:
for row,values in who_data.iterrows():
    cursor.execute(insert_who_dim,list(values))

In [332]:
insert_state_dim = 'INSERT INTO COVID_DATA.STAGE.STATE_DIM(REGION_NAME,LAT,LON) VALUES(%s,%s,%s)'

In [333]:
for row,value in state_data.iterrows():
    cursor.execute(insert_state_dim,list(value))

In [334]:
execute_dim_data_ins_sp = cursor.execute('call lkp_ins_dim_data()')

In [341]:
# insert_Fact_Table_Query = '''insert into covid_data.covid_dataset_2019.cases_data_fact(COUNTRY_ID,STATE_ID,WREGION_ID,DATE_ID,CONFIRMED,DEATHS,RECOVERED) select cd.country_id,sd.state_id,wd.wregion_id,dm.date_id,CAST(fd.confirmed AS INT),CAST(fd.deaths AS INT),CAST(fd.recovered AS INT) from covid_data.stage.file_data fd left join covid_data.covid_dataset_2019.date_dim dm on dm.date = fd.date left join covid_data.covid_dataset_2019.country_dim cd on cd.country_name = REPLACE(fd.country,'*','') left join covid_data.covid_dataset_2019.state_dim sd on sd.region_name = fd.state left join covid_data.covid_dataset_2019.who_dim wd on wd.region_name = fd.who_region left join covid_data.covid_dataset_2019.cases_data_fact cccd on cccd.country_id = cd.country_id and cccd.date_id = dm.date_id and cccd.wregion_id = wd.wregion_id and coalesce(cccd.state_id,0) = coalesce(sd.state_id,0) and cccd.confirmed = CAST(fd.confirmed AS INT) and cccd.deaths = CAST(fd.deaths AS INT) and cccd.recovered = CAST(fd.recovered AS INT) where cccd.country_id is null'''
# cursor.execute(insert_Fact_Table_Query)
execute_fact_table_ins_sp = cursor.execute('call ins_fact_table()')